In [1]:
import win32evtlog
import sys
from xml.etree import ElementTree as ET

In [26]:
evt_level_dict = {0 : 'LogAlways',
                1 : 'Critical',
                2 : 'Error',
                3 : 'Warning',
                4 : 'Informational',
                5 : 'Verbose'}

evt_opcode_dict = { 0 : 'Info',
                1: 'Start',
                2: 'Stop',
                3: 'DataCollectionStart',
                4: 'DataCollectionStop',
                5: 'Extension',
                6: 'Reply',
                7: 'Resume',
                8: 'Suspend',
                9: 'Send',
                240: 'Receive'}


def parse_XML_log(event):
    tree = ET.ElementTree(ET.fromstring(event))
    root = tree.getroot()
    ns = "{http://schemas.microsoft.com/win/2004/08/events/event}"
    data = {}
    for eventID in root.findall(".//"):
        if eventID.tag == f"{ns}System":
            for e_id in eventID.iter():
                if e_id.tag == f"{ns}System":
                    pass
                elif e_id.tag == f"{ns}Provider":
                    data["Provider"] = e_id.attrib.get('Name')
                elif e_id.tag == f"{ns}TimeCreated":
                    data["TimeCreated"] = e_id.attrib.get('SystemTime')
                elif e_id.tag == f"{ns}Correlation":
                    data["ActivityID"] = e_id.attrib.get('ActivityID')
                elif e_id.tag == f"{ns}Execution":
                    data["ProcessID"] = e_id.attrib.get('ProcessID')
                    data["ThreadID"] = e_id.attrib.get('ThreadID')
                elif e_id.tag == f"{ns}Level":
                    if not int(e_id.text) in evt_level_dict.keys():
                        data['Level'] = "unknown"
                    else:
                        data['Level'] = evt_level_dict[int(e_id.text)]
                elif e_id.tag == f"{ns}Opcode":
                    if not int(e_id.text) in evt_opcode_dict.keys():
                        data['Opcode'] = "unknown"
                    else:
                        data['Opcode'] = evt_opcode_dict[int(e_id.text)]
                else:
                    att = e_id.tag.replace(f"{ns}", "")
                    data[att] = e_id.text

        if eventID.tag == f"{ns}EventData":
            for attr in eventID.iter():
                if attr.tag == f'{ns}Data':
                    if attr.get('Name') is None:
                        data["Data"] = attr.text
                    else:
                        data[attr.get('Name')] = attr.text
                elif attr.tag == f'{ns}Binary':
                    data["Binary"] = attr.text

    return data

In [6]:
event_context = { "info": "this object is always passed to your callback" }
# Event log source to listen to
event_source = 'application'

def new_logs_event_handler(reason, context, evt):
    """
    Called when new events are logged.

    reason - reason the event was logged?
    context - context the event handler was registered with
    evt - event handle
    """
    # Just print some information about the event
    # print ('reason', reason, 'context', context, 'event handle', evt)

    event = win32evtlog.EvtRender(evt, win32evtlog.EvtRenderEventXml)
    result = " ".join(l.strip() for l in event.splitlines())
    print(result)
    with open('xml_logs.txt', 'a') as file:
        file.write(result)
        file.write('\n')
    # parse_XML_log(event=event)
    print(' - ')

    # Make sure all printed text is actually printed to the console now
    sys.stdout.flush()
    return 0

In [ ]:
# Subscribe to future events
subscription1 = win32evtlog.EvtSubscribe(event_source, win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription2 = win32evtlog.EvtSubscribe('system', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)
subscription3 = win32evtlog.EvtSubscribe('Security', win32evtlog.EvtSubscribeToFutureEvents, None, Callback=new_logs_event_handler, Context=event_context, Query=None)

<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='Microsoft-Windows-Security-Auditing' Guid='{54849625-5478-4994-a5ba-3e3b0328c30d}'/><EventID>4624</EventID><Version>2</Version><Level>0</Level><Task>12544</Task><Opcode>0</Opcode><Keywords>0x8020000000000000</Keywords><TimeCreated SystemTime='2023-06-04T04:39:03.4407224Z'/><EventRecordID>506098</EventRecordID><Correlation ActivityID='{0a4ce6a1-9699-0002-39e7-4c0a9996d901}'/><Execution ProcessID='860' ThreadID='7032'/><Channel>Security</Channel><Computer>DESKTOP-1LIHC4I</Computer><Security/></System><EventData><Data Name='SubjectUserSid'>S-1-5-18</Data><Data Name='SubjectUserName'>DESKTOP-1LIHC4I$</Data><Data Name='SubjectDomainName'>WORKGROUP</Data><Data Name='SubjectLogonId'>0x3e7</Data><Data Name='TargetUserSid'>S-1-5-18</Data><Data Name='TargetUserName'>SYSTEM</Data><Data Name='TargetDomainName'>NT AUTHORITY</Data><Data Name='TargetLogonId'>0x3e7</Data><Data Name='LogonType'>5</Data><Data Na

In [ ]:
data = []
with open("xml_logs.txt", "r+") as file1:
    lines = file1.readlines()   
    for line in lines:              
        data.append(parse_XML_log(line))   
        
data

In [ ]:
type(data[0])

dict

<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='Microsoft-Windows-Security-Auditing' Guid='{54849625-5478-4994-a5ba-3e3b0328c30d}'/><EventID>4798</EventID><Version>0</Version><Level>0</Level><Task>13824</Task><Opcode>0</Opcode><Keywords>0x8020000000000000</Keywords><TimeCreated SystemTime='2023-06-04T08:35:46.6082574Z'/><EventRecordID>506827</EventRecordID><Correlation ActivityID='{0a4ce6a1-9699-0002-39e7-4c0a9996d901}'/><Execution ProcessID='860' ThreadID='608'/><Channel>Security</Channel><Computer>DESKTOP-1LIHC4I</Computer><Security/></System><EventData><Data Name='TargetUserName'>Louki</Data><Data Name='TargetDomainName'>DESKTOP-1LIHC4I</Data><Data Name='TargetSid'>S-1-5-21-2384755564-1941788603-3002086374-1001</Data><Data Name='SubjectUserSid'>S-1-5-21-2384755564-1941788603-3002086374-1001</Data><Data Name='SubjectUserName'>Louki</Data><Data Name='SubjectDomainName'>DESKTOP-1LIHC4I</Data><Data Name='SubjectLogonId'>0x2d243</Data><Data Na

In [ ]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Windows_Event_Logs']
collection = db['mycollection']


# Insert the list into the collection

collection.insert_many(data)

# Print the inserted documents
for doc in collection.find():
    print(doc)